# Data Cleaning Drill

Determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [83]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import difflib
import itertools
import Levenshtein
import nltk
from nltk.corpus import stopwords
import string
import scipy.stats
import statistics

In [59]:
def sim_pairs(list):
    for pair in itertools.combinations(set(list),2):
        similarity = fuzz.partial_ratio(pair[0],pair[1])
        if similarity >= 90:
            print(pair, round(similarity,2))
            
def sim_pairs2(list):
    for pair in itertools.combinations(set(list),2):
        similarity = fuzz.token_sort_ratio(pair[0],pair[1])
        if similarity >= 90:
            print(pair, round(similarity,2))

def text_consolidate(list,df,sensitivity):
    for pair in itertools.combinations(set(list),2):
        similarity = fuzz.partial_ratio(pair[0],pair[1])
        if similarity >= sensitivity:
            df.loc[df.str.contains(pair[1])] = pair[0]
            
def text_consolidate2(list,df,sensitivity):
    for pair in itertools.combinations(set(list),2):
        similarity = fuzz.token_sort_ratio(pair[0],pair[1])
        if similarity >= sensitivity:
            df.loc[df.str.contains(pair[1])] = pair[0]
            
            

In [60]:
df = pd.read_csv('WELLCOME2.csv',encoding='latin1')

In [61]:
df = df.replace(np.nan,'',regex=True)

In [62]:
df = df.rename(columns={'PMID/PMCID': 'pmid/pmcid', 
                        'Publisher': 'publisher',
                        'Journal title': 'journal_title',
                        'Article title': 'article_title',
                        'COST (£) charged to Wellcome (inc VAT when charged)': 'wellcome_charge',
                        })

In [63]:
df['pmid/pmcid'] = df['pmid/pmcid'].str.extract('(\d+)')

/Users/wliao/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [64]:
df['publisher'] = df['publisher'].str.strip()
df['publisher'] = df['publisher'].str.lower()
df['publisher'] = df['publisher'].str.replace('[^\w\s]','')
df['journal_title'] = df['journal_title'].str.strip()
df['journal_title'] = df['journal_title'].str.lower()
df['journal_title'] = df['journal_title'].str.replace('[^\w\s]','')

In [65]:
df['wellcome_charge'] = df['wellcome_charge'].replace({'\£':''}, regex = True)

In [66]:
df['wellcome_charge'] = pd.to_numeric(df['wellcome_charge'],errors='coerce')

In [67]:
df['wellcome_charge'] = scipy.stats.mstats.winsorize(df['wellcome_charge'],limits=[0.05,0.05])

In [68]:
df.loc[df['publisher'].str.contains('bmc')] = 'biomed central'
#df.loc[df['publisher'].str.contains('oup')] = 'oxford university press'
df.loc[df['publisher'].str.contains('plos')] = 'public library of science'
#df.loc[df['publisher'].str.contains('wlieyblackwell')] = 'wiley'

incorrect_to_correct = {'acs publications':'american chemical society publications','bmc':'biomed central', 'oup':'oxford university press', 'plos':'public library of science','acs':'american chemical society'}
df.publisher = df.publisher.replace(incorrect_to_correct)

In [69]:
df.head(15)

,pmid/pmcid,publisher,journal_title,article_title,wellcome_charge
0,NaN,cup,psychological medicine,Reduced parahippocampal cortical thickness in ...,741.35
1,3679557,american chemical society,biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04
2,23043264,american chemical society,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",741.35
3,23438330,american chemical society,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,741.35
4,23438216,american chemical society,j org chem,Regioselective opening of myo-inositol orthoes...,741.35
5,3579457,american chemical society,journal of medicinal chemistry,Comparative Structural and Functional Studies ...,2392.2
6,3709265,american chemical society,journal of proteome research,Mapping Proteolytic Processing in the Secretom...,2367.95
7,23057412,american chemical society,mol pharm,Quantitative silencing of EGFP reporter gene b...,741.35
8,3780468,acs amercian chemical society publications,acs chemical biology,A Novel Allosteric Inhibitor of the Uridine Di...,1294.59
9,3621575,acs amercian chemical society publications,acs chemical biology,Chemical proteomic analysis reveals the drugab...,1294.78


In [70]:
text_consolidate(df['publisher'],df['publisher'],90)

In [71]:
text_consolidate2(df['journal_title'],df['journal_title'],90)

In [72]:
df['journal_title'].value_counts() 

plos  one                                                                170
biologicial chemistry                                                     66
neuroimage                                                                31
public library of science                                                 31
proceddings of the national academy of sciences of usa                    29
nucleic acid research                                                     29
biomed central                                                            28
plos neglected tropical diseases                                          23
plos pathogens                                                            22
plos genetics                                                             20
nature communications                                                     19
human molecular genetics                                                  19
biochemical journals                                                      15

In [78]:
plos_one = df[df['journal_title'].str.contains("plos  one")]
neuroimage = df[df['journal_title'].str.contains("neuroimage")]
nar = df[df['journal_title'].str.contains("nucleic acid research")]
bc = df[df['journal_title'].str.contains("biologicial chemistry")]
nas = df[df['journal_title'].str.contains("proceddings of the national academy of sciences of usa")]

In [79]:
print(plos_one['wellcome_charge'].mean())
print(neuroimage['wellcome_charge'].mean())
print(nar['wellcome_charge'].mean())
print(bc['wellcome_charge'].mean())
print(nas['wellcome_charge'].mean())

1087.09717647
2057.31805556
1164.50689655
1465.31257576
854.59137931


In [81]:
print(plos_one['wellcome_charge'].std())
print(neuroimage['wellcome_charge'].std())
print(nar['wellcome_charge'].std())
print(bc['wellcome_charge'].std())
print(nas['wellcome_charge'].std())

644.53595114
466.871610414
439.928346179
567.752152633
366.51309684


In [93]:
print(statistics.median(plos_one['wellcome_charge']))
print(statistics.median(neuroimage['wellcome_charge']))
print(statistics.median(nar['wellcome_charge']))
print(statistics.median(bc['wellcome_charge']))
print(statistics.median(nas['wellcome_charge']))

896.6400000000001
2289.245
852.0
1299.74
751.84


In [ ]:
sim_pairs(df['publisher'])

In [ ]:
df['journal_title'].value_counts() 

In [ ]:
sim_pairs2(df['journal_title'])